In [ ]:
# Install necessary
%%Capture
#! pip instal
#

In [77]:
import numpy as np
import pandas as pd
import re
import pandas as pd
import chardet.universaldetector
from IPython.display import display

# Visualization
from rich import print
from pprint import pprint
from rich.progress import track
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
import plotly
import plotly.express as px

# Import geo services
import geopy
from geopy import distance
from geopy.geocoders import Bing  # library import
geolocator = Bing(api_key="ArAEkjZybNV7puDe4lgO9FsX8VssJ57er2SG0SvcuN3YxL0bZ5U9wZUtPlddLrVx")
#import geocoder
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

# Loading special module for data split
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk("/Data"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip freeze > requirements.txt  # fixing versions of packages

In [24]:
RANDOM_SEED = 42  # fixing random seed for repeatability of experiment
DATA_DIR = "Data/"

In [26]:
# Checking dataset encoding
detector = chardet.UniversalDetector()
with open(DATA_DIR + "hotels_train.csv", "rb") as fh:
    for line in fh:
        detector.feed(line)
        if detector.done:
            break
detector.close()

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}

In [28]:
# Loading data
df_train = pd.read_csv(DATA_DIR+"hotels_train.csv", encoding="ascii") # dataset for training
df_test = pd.read_csv(DATA_DIR+"hotels_test.csv", encoding="ascii") # dataset for prediction
sample_submission = pd.read_csv(DATA_DIR+"/submission.csv", encoding="ascii") # submission

In [29]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

In [30]:
df_train.head(2)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097


In [31]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128935 entries, 0 to 128934
Data columns (total 16 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               128935 non-null  object 
 1   additional_number_of_scoring                128935 non-null  int64  
 2   review_date                                 128935 non-null  object 
 3   average_score                               128935 non-null  float64
 4   hotel_name                                  128935 non-null  object 
 5   reviewer_nationality                        128935 non-null  object 
 6   negative_review                             128935 non-null  object 
 7   review_total_negative_word_counts           128935 non-null  int64  
 8   total_number_of_reviews                     128935 non-null  int64  
 9   positive_review                             128935 non-null  object 
 

In [32]:
df_test.head(2)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,tags,days_since_review,lat,lng
0,Via Senigallia 6 20161 Milan Italy,904,7/21/2017,8.1,Hotel Da Vinci,United Kingdom,Would have appreciated a shop in the hotel th...,52,16670,Hotel was great clean friendly staff free bre...,62,1,"[' Leisure trip ', ' Couple ', ' Double Room '...",13 days,45.533137,9.171102
1,Arlandaweg 10 Westpoort 1043 EW Amsterdam Neth...,612,12/12/2016,8.6,Urban Lodge Hotel,Belgium,No tissue paper box was present at the room,10,5018,No Positive,0,7,"[' Leisure trip ', ' Group ', ' Triple Room ',...",234 day,52.385649,4.834443


In [33]:
sample_submission.head(2)

,reviewer_score,id
0,1,488440
1,10,274649


In [34]:
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128935 entries, 0 to 128934
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype
---  ------          --------------   -----
 0   reviewer_score  128935 non-null  int64
 1   id              128935 non-null  int64
dtypes: int64(2)
memory usage: 2.0 MB


In [93]:
# Joining train and test data for feature engineering

df_train["sample"] = 1 # train mark
df_test["sample"] = 0 # test mark
df_test["reviewer_score"] = 0 # we have to predict 'reviewer_score', so now it would be filled with 0 in test data

data = pd.concat([df_train, df_test], sort=False).reset_index(drop=True) # join

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 18 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               515738 non-null  object 
 1   additional_number_of_scoring                515738 non-null  int64  
 2   review_date                                 515738 non-null  object 
 3   average_score                               515738 non-null  float64
 4   hotel_name                                  515738 non-null  object 
 5   reviewer_nationality                        515738 non-null  object 
 6   negative_review                             515738 non-null  object 
 7   review_total_negative_word_counts           515738 non-null  int64  
 8   total_number_of_reviews                     515738 non-null  int64  
 9   positive_review                             515738 non-null  object 
 

In [37]:
data.describe()

,additional_number_of_scoring,average_score,review_total_negative_word_counts,total_number_of_reviews,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,lat,lng,sample
count,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,512470.000000,512470.000000,515738.000000
mean,498.081836,8.397487,18.539450,2743.743944,17.776458,7.166001,6.297672,49.442439,2.823803,0.749999
std,500.538467,0.548048,29.690831,2317.464868,21.804185,11.040228,3.902295,3.466325,4.579425,0.433014
min,1.000000,5.200000,0.000000,43.000000,0.000000,1.000000,0.000000,41.328376,-0.369758,0.000000
25%,169.000000,8.100000,2.000000,1161.000000,5.000000,1.000000,0.625000,48.214662,-0.143372,0.250000
50%,341.000000,8.400000,9.000000,2134.000000,11.000000,3.000000,7.900000,51.499981,0.010607,1.000000
75%,660.000000,8.800000,23.000000,3613.000000,22.000000,8.000000,9.600000,51.516288,4.834443,1.000000
max,2682.000000,9.800000,408.000000,16670.000000,395.000000,355.000000,10.000000,52.400181,16.429233,1.000000


In [38]:
data.describe(include="object")

,hotel_address,review_date,hotel_name,reviewer_nationality,negative_review,positive_review,tags,days_since_review
count,515738,515738,515738,515738,515738,515738,515738,515738
unique,1493,731,1492,227,330011,412601,55242,731
top,163 Marsh Wall Docklands Tower Hamlets London ...,8/2/2017,Britannia International Hotel Canary Wharf,United Kingdom,No Negative,No Positive,"[' Leisure trip ', ' Couple ', ' Double Room '...",1 days
freq,4789,2585,4789,245246,127890,35946,5101,2585


### We will divide our work with data into two parts:<br>
 * ##### data related to hotels("hotel_address", "average_score", "hotel_name", "lat", "lng")
 * ##### data related to reviewers(the rest features)

### "The Features Of The Hotels"
Let's look at missing data in **latitudes** and **longitudes** of hotels geo positions:

In [73]:
def get_nan_cols(df):
    """"Counting missing data in dataframe by feature in percents"""
    features_null_percent = df.isnull().mean().round(5) * 100
    cols_with_null = features_null_percent[features_null_percent>0].sort_values(ascending=False)
    display(cols_with_null)

In [74]:
print("[cyan]Percents of missing data[/cyan]")
get_nan_cols(data)
num = data[(data["lat"].isna()) | (data["lng"].isna())].shape[0]
print(F"[cyan]Number if rows with missing data[/cyan]: [cyan bold]{num}[/cyan bold]")

Percents of missing data

lat    0.634
lng    0.634
dtype: float64

Number if rows with missing data: 3268

In [81]:
hotels_null = data.loc[(data["lat"].isna()) | (data["lng"].isna()),
                       ["hotel_address", "hotel_name", "lat", "lng"]].drop_duplicates()
print("[cyan]Unique hotels with missing data[/cyan]")
display(hotels_null)  # Looking at unique hotels with missing data
# As I tried a lot of variants, this one is the most effective,
# but long enough for fixing data for correct geocoding
def address_repair(address):
    """Fixing values in hotel_address"""
    address = address.replace("tra e", "traße").replace("r nen", "rünen")\
    .replace("st dt", "städt").replace("mr mo", "mremo")\
    .replace("Sep lveda", "C/Sepulveda").replace("Eixample", " ")\
    .replace("W hringer", "Währinger Gürtel").replace("G rtel", "Gürtel")\
    .replace("D bling", " ").replace("P pini re", "Pépinière")\
    .replace("Ga t", "Gaîté").replace("Bail n", "Bailén")\
    .replace("Clar s", "Claris")
    return address

hotels_null["hotel_address"] = hotels_null["hotel_address"].apply(address_repair) # repairing hotel addresses
#hotels_null["lat"] = hotels_null["hotel_address"].apply(                        # latitude fixing
#                                    lambda x: geolocator.geocode(x).latitude)
#hotels_null["lng"] = hotels_null["hotel_address"].apply(                        # longitude fixing
#                                    lambda x: geolocator.geocode(x).longitude)

Unique hotels with missing data

,hotel_address,hotel_name,lat,lng
122,Savoyenstra e 2 16 Ottakring 1160 Vienna Austria,Austria Trend Hotel Schloss Wilhelminenberg Wien,NaN,NaN
566,23 Rue Damr mont 18th arr 75018 Paris France,Holiday Inn Paris Montmartre,NaN,NaN
724,Josefst dter Stra e 10 12 08 Josefstadt 1080 V...,Fleming s Selection Hotel Wien City,NaN,NaN
754,W hringer Stra e 33 35 09 Alsergrund 1090 Vien...,Hotel Atlanta,NaN,NaN
1137,4 rue de la P pini re 8th arr 75008 Paris France,Maison Albar Hotel Paris Op ra Diamond,NaN,NaN
1189,Sieveringer Stra e 4 19 D bling 1190 Vienna Au...,Derag Livinghotel Kaiser Franz Joseph Vienna,NaN,NaN
1193,Taborstra e 8 A 02 Leopoldstadt 1020 Vienna Au...,Hotel City Central,NaN,NaN
1568,Bail n 4 6 Eixample 08010 Barcelona Spain,NH Collection Barcelona Podium,NaN,NaN
1758,Gr nentorgasse 30 09 Alsergrund 1090 Vienna Au...,City Hotel Deutschmeister,NaN,NaN
2998,Landstra er G rtel 5 03 Landstra e 1030 Vienna...,Hotel Daniel Vienna,NaN,NaN


In [84]:
# Temporary data loading
#hotels_null = pd.read_csv(DATA_DIR + "long_lang.csv", index_col="Unnamed: 0")

In [94]:
data = data.merge(hotels_null, how="left", on="hotel_name") # adding filled data to main dataframe
data["lat"] = data["lat_x"].fillna(data["lat_y"]) # fixing latitude in main dataframe
data["lng"] = data["lng_x"].fillna(data["lng_y"]) # fixing longitude in main dataframe
data.drop(columns=["lat_x", "lng_x", "lat_y", "lng_y",    # dropping unnecessary columns
                   "hotel_address_y"], inplace=True)
data.rename(columns = {"hotel_address_x":"hotel_address"}, inplace = True)

#### Now let's look at unique hotels in all dataframe

In [108]:
hotels_unique = data[["hotel_address", "hotel_name", "lat", "lng"]].drop_duplicates()

# Let's look at the map
m_1 = folium.Map(tiles="openstreetmap", zoom_start=5, location=[48.779124, 9.180090]) # create the map

# Add points to the map
hotels = MarkerCluster()
for idx, row in hotels_unique.iterrows():
    hotels.add_child(Marker([row["lat"], row["lng"]]))
m_1.add_child(hotels)

# Display the map
m_1

In [103]:
hotels_null

,hotel_address,hotel_name,lat,lng
122,Savoyenstraße 2 16 Ottakring 1160 Vienna Austria,Austria Trend Hotel Schloss Wilhelminenberg Wien,48.219580,16.285510
566,23 Rue Damremont 18th arr 75018 Paris France,Holiday Inn Paris Montmartre,48.888876,2.333177
724,Josefstädter Straße 10 12 08 Josefstadt 1080 V...,Fleming s Selection Hotel Wien City,48.209390,16.353390
754,Währinger Gürtel Straße 33 35 09 Alsergrund 10...,Hotel Atlanta,48.220299,16.355846
1137,4 rue de la Pépinière 8th arr 75008 Paris France,Maison Albar Hotel Paris Op ra Diamond,48.875257,2.323369
1189,Sieveringer Straße 4 19 1190 Vienna Austria,Derag Livinghotel Kaiser Franz Joseph Vienna,48.246060,16.341010
1193,Taborstraße 8 A 02 Leopoldstadt 1020 Vienna Au...,Hotel City Central,48.213410,16.379880
1568,Bailén 4 6 08010 Barcelona Spain,NH Collection Barcelona Podium,41.391672,2.177913
1758,Grünentorgasse 30 09 Alsergrund 1090 Vienna Au...,City Hotel Deutschmeister,48.220820,16.366640
2998,Landstraßer Gürtel 5 03 Landstraße 1030 Vienna...,Hotel Daniel Vienna,48.188700,16.383710


In [79]:
#pivot_n = data[['hotel_address', 'hotel_name']].groupby(
#    by='hotel_name'
#).nunique("
#pivot_n[pivot_n['hotel_address'] != 1]

,hotel_address
hotel_name,
Hotel Regina,3


In [ ]:
#hotels['hotel_name'] = hotels.apply(
#    lambda row: (row['hotel_name'] + ' ' + row['hotel_address'].split()[-2])
#    if (row['hotel_name'] in name_rep)
#    else row['hotel_name'], axis=1)

In [13]:

print(colored('\nПроверим полученные изменения:', 'blue'))


Проверим полученные изменения:


In [15]:
# Choosing country name from 'hotel_address'

data["hotel_name"] = data.apply(
    lambda row: (row["hotel_name"]+ ", " +
                 #(re.findall(r".*[(\d+)].(\w*)", row["hotel_address"])[0]) + " " +
                 (re.findall(r".*[(\d+)].\w*.(\w*.{,100})", row["hotel_address"])[0])), axis=1)
    #if (x["lat"]==0 | x["lng"].isna())
    #else x["hotel_name"],

In [128]:
ln = "jnelknw 7846 wef 8746  sts od am"
(re.findall(r".*[(\d+)].\w*.(\w*.{,100})", ln)[0])

'sts od am'

In [ ]:
data.nunique(dropna=False)

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(), annot=True)

In [ ]:
# убираем признаки которые еще не успели обработать, 
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
data.drop(object_columns, axis = 1, inplace=True)

In [ ]:
data.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAPE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
list(sample_submission)

In [ ]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)